In [2]:
import torch
import torch.nn as nn
from torchinfo import summary

In [3]:


class TrajectoryLSTM(nn.Module):
    def __init__(self, input_dim=2+2+32, hidden_dim=128, output_dim=2, num_layers=1):
        super().__init__()
        self.encoder_lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.decoder_lstm = nn.LSTM(output_dim, hidden_dim, num_layers, batch_first=True)
        self.output_fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, input_seq, target_len=80):
        batch_size = input_seq.size(0)

        # Encode
        _, (h, c) = self.encoder_lstm(input_seq)

        # Decode
        decoder_input = input_seq[:, -1:, :2]  # just x, y of last input
        outputs = []

        for _ in range(target_len):
            out, (h, c) = self.decoder_lstm(decoder_input, (h, c))
            pred = self.output_fc(out)  # predict (x, y)
            outputs.append(pred)
            decoder_input = pred  # feed predicted position

        return torch.cat(outputs, dim=1)  # shape: (B, 80, 2)


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
import math


class TrajectoryLSTM(nn.Module):
    """
    LSTM model for trajectory prediction using MTR dataset format.
    
    Input: obj_trajs from MTR dataset with shape (num_center_objects, num_objects, num_timestamps, num_features)
    Output: Future trajectory predictions
    """
    
    def __init__(self, 
                 input_dim=29,  # Based on MTR dataset obj_trajs feature dimension
                 hidden_dim=256,
                 num_layers=2,
                 num_modes=6,  # Number of prediction modes
                 future_steps=80,  # Number of future timesteps to predict
                 dropout=0.1):
        super(TrajectoryLSTM, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.num_modes = num_modes
        self.future_steps = future_steps
        self.dropout = dropout
        
        # Feature encoder for input trajectories
        self.feature_encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim)
        )
        
        # LSTM for temporal modeling
        self.lstm = nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=False
        )
        
        # Multi-modal prediction heads
        self.mode_predictor = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_modes)
        )
        
        # Trajectory decoder for each mode
        self.traj_decoders = nn.ModuleList([
            nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_dim, hidden_dim),
                nn.ReLU(),
                nn.Linear(hidden_dim, future_steps * 4)  # x, y, vx, vy for each timestep
            ) for _ in range(num_modes)
        ])
        
        # Attention mechanism for object interactions
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=8,
            dropout=dropout,
            batch_first=True
        )
        
        #loss function
        self.criterion = self.TrajectoryLoss()
        self._init_weights()
    
    def _init_weights(self):
        """Initialize model weights"""
        for name, param in self.named_parameters():
            if 'weight' in name:
                if len(param.shape) >= 2:
                    nn.init.xavier_uniform_(param)
                else:
                    nn.init.uniform_(param, -0.1, 0.1)
            elif 'bias' in name:
                nn.init.constant_(param, 0)
    
    class TrajectoryLoss(nn.Module):
        """
        Loss function for trajectory prediction
        """
        
        def __init__(self, 
                    regression_loss_weight=1.0,
                    classification_loss_weight=1.0,
                    future_loss_weight=1.0):
            super().__init__()
            self.reg_weight = regression_loss_weight
            self.cls_weight = classification_loss_weight
            self.future_weight = future_loss_weight
        
        def forward(self, pred_scores, pred_trajs, batch_dict):
            """
            Compute loss
            
            Args:
                pred_scores: (batch_size, num_modes)
                pred_trajs: (batch_size, num_modes, future_steps, 4)
                batch_dict: Contains ground truth data
            
            Returns:
                loss_dict: Dictionary containing different loss components
            """
            center_gt_trajs = batch_dict['input_dict']['center_gt_trajs'].to('cuda')  # (batch_size, future_steps, 4)
            center_gt_trajs_mask = batch_dict['input_dict']['center_gt_trajs_mask'].to('cuda')  # (batch_size, future_steps)
            
            batch_size, num_modes, future_steps, _ = pred_trajs.shape
            
            # Compute trajectory regression loss for each mode
            gt_trajs_expanded = center_gt_trajs.unsqueeze(1).expand(-1, num_modes, -1, -1)
            gt_mask_expanded = center_gt_trajs_mask.unsqueeze(1).expand(-1, num_modes, -1)
            
            # L2 loss for position (x, y)
            pos_loss = F.mse_loss(
                pred_trajs[:, :, :, :2] * gt_mask_expanded.unsqueeze(-1),
                gt_trajs_expanded[:, :, :, :2] * gt_mask_expanded.unsqueeze(-1),
                reduction='none'
            ).sum(dim=-1)  # (batch_size, num_modes, future_steps)
            
            # L2 loss for velocity (vx, vy)
            vel_loss = F.mse_loss(
                pred_trajs[:, :, :, 2:4] * gt_mask_expanded.unsqueeze(-1),
                gt_trajs_expanded[:, :, :, 2:4] * gt_mask_expanded.unsqueeze(-1),
                reduction='none'
            ).sum(dim=-1)  # (batch_size, num_modes, future_steps)
            
            # Weighted loss over time (give more weight to near future)
            time_weights = torch.exp(-0.1 * torch.arange(future_steps, device=pred_trajs.device))
            time_weights = time_weights.view(1, 1, -1)
            
            pos_loss = (pos_loss * time_weights * gt_mask_expanded).sum(dim=-1)  # (batch_size, num_modes)
            vel_loss = (vel_loss * time_weights * gt_mask_expanded).sum(dim=-1)  # (batch_size, num_modes)
            
            # Find best mode for each sample
            total_traj_loss = pos_loss + vel_loss  # (batch_size, num_modes)
            best_mode_indices = torch.argmin(total_traj_loss, dim=1)  # (batch_size,)
            
            # Regression loss (best mode)
            best_pos_loss = pos_loss[torch.arange(batch_size), best_mode_indices].mean()
            best_vel_loss = vel_loss[torch.arange(batch_size), best_mode_indices].mean()
            regression_loss = best_pos_loss + best_vel_loss
            
            # Classification loss (encourage higher confidence for best mode)
            target_scores = torch.zeros_like(pred_scores)
            target_scores[torch.arange(batch_size), best_mode_indices] = 1.0
            classification_loss = F.cross_entropy(pred_scores, target_scores)
            
            # Total loss
            total_loss = (self.reg_weight * regression_loss + 
                        self.cls_weight * classification_loss)
            
            loss_dict = {
                'total_loss': total_loss,
                'regression_loss': regression_loss,
                'classification_loss': classification_loss,
                'pos_loss': best_pos_loss,
                'vel_loss': best_vel_loss
            }
            
            return loss_dict
    
    def forward(self, batch_dict):
        """
        Forward pass of the model
        
        Args:
            batch_dict: Dictionary containing:
                - obj_trajs: (batch_size, num_objects, num_timestamps, input_dim)
                - obj_trajs_mask: (batch_size, num_objects, num_timestamps)
                - track_index_to_predict: (batch_size,) indices of center objects
        
        Returns:
            pred_scores: (batch_size, num_modes) - confidence scores for each mode
            pred_trajs: (batch_size, num_modes, future_steps, 4) - predicted trajectories
        """
        input_dict=batch_dict["input_dict"]
        obj_trajs = input_dict['obj_trajs'].to("cuda")  # (batch_size, num_objects, num_timestamps, input_dim)
        obj_trajs_mask = input_dict['obj_trajs_mask'].to("cuda")  # (batch_size, num_objects, num_timestamps)
        track_indices = input_dict['track_index_to_predict'].to("cuda")  # (batch_size,)
        # map_polylines, map_polylines_mask = input_dict['map_polylines'].to("cuda"), input_dict['map_polylines_mask'].to("cuda") # (num_center_objects, num_topk_polylines, num_points_each_polyline, 9): [x, y, z, dir_x, dir_y, dir_z, global_type, pre_x, pre_y]

        

        
        batch_size, num_objects, num_timestamps, input_dim = obj_trajs.shape
        
        # Encode input features
        obj_features = self.feature_encoder(obj_trajs.view(-1, input_dim))
        obj_features = obj_features.view(batch_size, num_objects, num_timestamps, self.hidden_dim)
        
        # Apply mask to features
        mask_expanded = obj_trajs_mask.unsqueeze(-1).expand(-1, -1, -1, self.hidden_dim)
        obj_features = obj_features * mask_expanded.float()
        
        # Process each object's trajectory through LSTM
        all_lstm_outputs = []
        
        for obj_idx in range(num_objects):
            obj_seq = obj_features[:, obj_idx, :, :]  # (batch_size, num_timestamps, hidden_dim)
            lstm_out, _ = self.lstm(obj_seq)  # (batch_size, num_timestamps, hidden_dim)
            
            # Take the last valid output for each sequence
            seq_lengths = obj_trajs_mask[:, obj_idx, :].sum(dim=1)  # (batch_size,)
            last_outputs = []
            for b in range(batch_size):
                if seq_lengths[b] > 0:
                    last_idx = int(seq_lengths[b] - 1)
                    last_outputs.append(lstm_out[b, last_idx, :])
                else:
                    last_outputs.append(torch.zeros(self.hidden_dim, device=obj_seq.device))
            
            last_output = torch.stack(last_outputs, dim=0)  # (batch_size, hidden_dim)
            all_lstm_outputs.append(last_output)
        
        all_lstm_outputs = torch.stack(all_lstm_outputs, dim=1)  # (batch_size, num_objects, hidden_dim)
        
        # Apply attention mechanism for object interactions
        attn_output, _ = self.attention(
            all_lstm_outputs, all_lstm_outputs, all_lstm_outputs,
            key_padding_mask=~(obj_trajs_mask.sum(dim=2) > 0)  # (batch_size, num_objects)
        )
        
        # Extract center object features
        center_features = []
        for b in range(batch_size):
            center_idx = track_indices[b]
            center_features.append(attn_output[b, center_idx, :])
        center_features = torch.stack(center_features, dim=0)  # (batch_size, hidden_dim)
        
        # Predict mode probabilities
        mode_logits = self.mode_predictor(center_features)  # (batch_size, num_modes)
        pred_scores = F.softmax(mode_logits, dim=-1)
        
        # Predict trajectories for each mode
        pred_trajs_list = []
        for mode_idx in range(self.num_modes):
            traj_flat = self.traj_decoders[mode_idx](center_features)  # (batch_size, future_steps * 4)
            traj = traj_flat.view(batch_size, self.future_steps, 4)  # (batch_size, future_steps, 4)
            pred_trajs_list.append(traj)
        
        pred_trajs = torch.stack(pred_trajs_list, dim=1)  # (batch_size, num_modes, future_steps, 4)
        
        
        # Compute loss
        loss_dict = self.criterion(pred_scores, pred_trajs, batch_dict)

        return loss_dict, pred_scores, pred_trajs

        return pred_scores, pred_trajs


class TrajectoryLoss(nn.Module):
    """
    Loss function for trajectory prediction
    """
    
    def __init__(self, 
                 regression_loss_weight=1.0,
                 classification_loss_weight=1.0,
                 future_loss_weight=1.0):
        super(TrajectoryLoss, self).__init__()
        self.reg_weight = regression_loss_weight
        self.cls_weight = classification_loss_weight
        self.future_weight = future_loss_weight
    
    def forward(self, pred_scores, pred_trajs, batch_dict):
        """
        Compute loss
        
        Args:
            pred_scores: (batch_size, num_modes)
            pred_trajs: (batch_size, num_modes, future_steps, 4)
            batch_dict: Contains ground truth data
        
        Returns:
            loss_dict: Dictionary containing different loss components
        """
        center_gt_trajs = batch_dict['input_dict']['center_gt_trajs'].to('cuda')  # (batch_size, future_steps, 4)
        center_gt_trajs_mask = batch_dict['input_dict']['center_gt_trajs_mask'].to('cuda')  # (batch_size, future_steps)
        
        batch_size, num_modes, future_steps, _ = pred_trajs.shape
        
        # Compute trajectory regression loss for each mode
        gt_trajs_expanded = center_gt_trajs.unsqueeze(1).expand(-1, num_modes, -1, -1)
        gt_mask_expanded = center_gt_trajs_mask.unsqueeze(1).expand(-1, num_modes, -1)
        
        # L2 loss for position (x, y)
        pos_loss = F.mse_loss(
            pred_trajs[:, :, :, :2] * gt_mask_expanded.unsqueeze(-1),
            gt_trajs_expanded[:, :, :, :2] * gt_mask_expanded.unsqueeze(-1),
            reduction='none'
        ).sum(dim=-1)  # (batch_size, num_modes, future_steps)
        
        # L2 loss for velocity (vx, vy)
        vel_loss = F.mse_loss(
            pred_trajs[:, :, :, 2:4] * gt_mask_expanded.unsqueeze(-1),
            gt_trajs_expanded[:, :, :, 2:4] * gt_mask_expanded.unsqueeze(-1),
            reduction='none'
        ).sum(dim=-1)  # (batch_size, num_modes, future_steps)
        
        # Weighted loss over time (give more weight to near future)
        time_weights = torch.exp(-0.1 * torch.arange(future_steps, device=pred_trajs.device))
        time_weights = time_weights.view(1, 1, -1)
        
        pos_loss = (pos_loss * time_weights * gt_mask_expanded).sum(dim=-1)  # (batch_size, num_modes)
        vel_loss = (vel_loss * time_weights * gt_mask_expanded).sum(dim=-1)  # (batch_size, num_modes)
        
        # Find best mode for each sample
        total_traj_loss = pos_loss + vel_loss  # (batch_size, num_modes)
        best_mode_indices = torch.argmin(total_traj_loss, dim=1)  # (batch_size,)
        
        # Regression loss (best mode)
        best_pos_loss = pos_loss[torch.arange(batch_size), best_mode_indices].mean()
        best_vel_loss = vel_loss[torch.arange(batch_size), best_mode_indices].mean()
        regression_loss = best_pos_loss + best_vel_loss
        
        # Classification loss (encourage higher confidence for best mode)
        target_scores = torch.zeros_like(pred_scores)
        target_scores[torch.arange(batch_size), best_mode_indices] = 1.0
        classification_loss = F.cross_entropy(pred_scores, target_scores)
        
        # Total loss
        total_loss = (self.reg_weight * regression_loss + 
                     self.cls_weight * classification_loss)
        
        loss_dict = {
            'total_loss': total_loss,
            'regression_loss': regression_loss,
            'classification_loss': classification_loss,
            'pos_loss': best_pos_loss,
            'vel_loss': best_vel_loss
        }
        
        return loss_dict


def create_dataloader(dataset, batch_size=32, shuffle=True, num_workers=4):
    """
    Create DataLoader for the MTR dataset
    """
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        collate_fn=collate_batch
    )


def collate_batch(batch_list):
    """
    Custom collate function to handle variable number of objects per scene
    """
    batch_dict = {}
    
    # Stack all the tensors
    for key in batch_list[0].keys():
        if isinstance(batch_list[0][key], np.ndarray):
            batch_dict[key] = torch.from_numpy(np.stack([item[key] for item in batch_list], axis=0))
        elif isinstance(batch_list[0][key], torch.Tensor):
            batch_dict[key] = torch.stack([item[key] for item in batch_list], axis=0)
        else:
            batch_dict[key] = [item[key] for item in batch_list]
    
    # Add batch size info
    batch_dict['batch_size'] = len(batch_list)
    batch_dict['batch_sample_count'] = [1] * len(batch_list)  # Each sample is one center object
    
    return batch_dict


def train_model(model, train_dataloader, val_dataloader, num_epochs=100, lr=1e-3):
    """
    Training loop for the LSTM model
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)
    criterion = TrajectoryLoss()
    
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_losses = []
        
        for batch_idx, batch_dict in enumerate(train_dataloader):
            # Move data to device
            for key, value in batch_dict.items():
                if isinstance(value, torch.Tensor):
                    batch_dict[key] = value.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            pred_scores, pred_trajs = model(batch_dict)
            
            # Compute loss
            loss_dict = criterion(pred_scores, pred_trajs, batch_dict)
            loss = loss_dict['total_loss']
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_losses.append(loss.item())
            
            if batch_idx % 100 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item():.4f}')
        
        # Validation phase
        model.eval()
        val_losses = []
        
        with torch.no_grad():
            for batch_dict in val_dataloader:
                # Move data to device
                for key, value in batch_dict.items():
                    if isinstance(value, torch.Tensor):
                        batch_dict[key] = value.to(device)
                
                pred_scores, pred_trajs = model(batch_dict)
                loss_dict = criterion(pred_scores, pred_trajs, batch_dict)
                val_losses.append(loss_dict['total_loss'].item())
        
        scheduler.step()
        
        avg_train_loss = np.mean(train_losses)
        avg_val_loss = np.mean(val_losses)
        
        print(f'Epoch {epoch}: Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), '/code/jjiang23/csc587/KimchiVision/best_trajectory_lstm.pth')
    
    return model



In [2]:
from mtr.datasets import build_dataloader
from mtr.config import cfg, cfg_from_yaml_file
from mtr.utils import common_utils

In [ ]:
cfg_from_yaml_file("/code/jjiang23/csc587/KimchiVision/cfg/kimchiConfig.yaml", cfg)
logger = common_utils.create_logger("/files/waymo/log.txt", rank=0)
from easydict import EasyDict as edict
args = edict({
    "batch_size": 32,
    "workers": 4,
    "merge_all_iters_to_one_epoch": False,
    "epochs": 1,
    "add_worker_init_fn": False,
})
train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    batch_size=args.batch_size,
    dist=False, workers=args.workers,
    logger=logger,
    training=True,
    merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
    total_epochs=args.epochs,
    add_worker_init_fn=args.add_worker_init_fn,
)
test_set, test_loader, sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG,
        batch_size=args.batch_size,
        dist=False, workers=args.workers, logger=logger, training=False
    )

NameError: name 'cfg_from_yaml_file' is not defined

In [3]:

# Initialize model
model = TrajectoryLSTM(input_dim=29, hidden_dim=256, num_modes=6)

# Train the model
trained_model = train_model(model, train_loader, test_loader)

NameError: name 'train_loader' is not defined

In [6]:
# Motion Transformer (MTR): https://arxiv.org/abs/2209.13508
# Published at NeurIPS 2022
# Written by Shaoshuai Shi 
# All Rights Reserved
#common libs
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torchinfo import summary
import math
from easydict import EasyDict as edict


import argparse
import datetime
import glob
import os
from pathlib import Path
import math

import torch
import torch.nn as nn
import torch.optim.lr_scheduler as lr_sched
from tensorboardX import SummaryWriter

from mtr.datasets import build_dataloader
from mtr.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from mtr.utils import common_utils


from train_utils.train_utils import train_model


def parse_config():
    # parser = argparse.ArgumentParser(description='arg parser')
    # parser.add_argument('--cfg_file', type=str, default=None, help='specify the config for training')

    # parser.add_argument('--batch_size', type=int, default=None, required=False, help='batch size for training')
    # parser.add_argument('--epochs', type=int, default=None, required=False, help='number of epochs to train for')
    # parser.add_argument('--workers', type=int, default=8, help='number of workers for dataloader')
    # parser.add_argument('--extra_tag', type=str, default='default', help='extra tag for this experiment')
    # parser.add_argument('--ckpt', type=str, default=None, help='checkpoint to start from')
    # parser.add_argument('--pretrained_model', type=str, default=None, help='pretrained_model')
    # parser.add_argument('--launcher', choices=['none', 'pytorch', 'slurm'], default='none')
    # parser.add_argument('--tcp_port', type=int, default=18888, help='tcp port for distrbuted training')
    # parser.add_argument('--without_sync_bn', action='store_true', default=False, help='whether to use sync bn')
    # parser.add_argument('--fix_random_seed', action='store_true', default=False, help='')
    # parser.add_argument('--ckpt_save_interval', type=int, default=2, help='number of training epochs')
    # parser.add_argument('--local_rank', type=int, default=None, help='local rank for distributed training')
    # parser.add_argument('--max_ckpt_save_num', type=int, default=5, help='max number of saved checkpoint')
    # parser.add_argument('--merge_all_iters_to_one_epoch', action='store_true', default=False, help='')
    # parser.add_argument('--set', dest='set_cfgs', default=None, nargs=argparse.REMAINDER,
    #                     help='set extra config keys if needed')

    # parser.add_argument('--max_waiting_mins', type=int, default=0, help='max waiting minutes')
    # parser.add_argument('--start_epoch', type=int, default=0, help='')
    # parser.add_argument('--save_to_file', action='store_true', default=False, help='')
    # parser.add_argument('--not_eval_with_train', action='store_true', default=False, help='')
    # parser.add_argument('--logger_iter_interval', type=int, default=50, help='')
    # parser.add_argument('--ckpt_save_time_interval', type=int, default=300, help='in terms of seconds')

    # parser.add_argument('--add_worker_init_fn', action='store_true', default=False, help='')
    # args = parser.parse_args()
    
    cfg_from_yaml_file("/code/jjiang23/csc587/KimchiVision/cfg/kimchiConfig.yaml", cfg)
    # take all default args
    args = edict({
    "batch_size": 32,
    "workers": 4,
    "merge_all_iters_to_one_epoch": False,
    "epochs": 1,
    "add_worker_init_fn": False,
    "extra_tag": 'default',
    "launcher": 'none',
    "tcp_port": 18888,
    "without_sync_bn": False,
    "fix_random_seed": False,
    "ckpt_save_interval": 2,
    "local_rank": None,
    "max_ckpt_save_num": 5,
    "set_cfgs": None,
    "max_waiting_mins": 0,
    "start_epoch": 0,
    "save_to_file": False,
    "not_eval_with_train": False,
    "logger_iter_interval": 50,
    "ckpt_save_time_interval": 300,
    "add_worker_init_fn": False,
    "pretrained_model": None,
    "ckpt": None,
    "cfg_file": None,
    "fix_random_seed": False,
    "extra_tag": 'default',


    
    
})
    return args, cfg


def build_optimizer(model, opt_cfg):
    if opt_cfg.OPTIMIZER == 'Adam':
        optimizer = torch.optim.Adam(
            [each[1] for each in model.named_parameters()],
            lr=opt_cfg.LR, weight_decay=opt_cfg.get('WEIGHT_DECAY', 0)
        )
    elif opt_cfg.OPTIMIZER == 'AdamW':
        optimizer = torch.optim.AdamW(model.parameters(), lr=opt_cfg.LR, weight_decay=opt_cfg.get('WEIGHT_DECAY', 0))
    else:
        assert False

    return optimizer


def build_scheduler(optimizer, dataloader, opt_cfg, total_epochs, total_iters_each_epoch, last_epoch):
    decay_steps = [x * total_iters_each_epoch for x in opt_cfg.get('DECAY_STEP_LIST', [5, 10, 15, 20])]
    def lr_lbmd(cur_epoch):
        cur_decay = 1
        for decay_step in decay_steps:
            if cur_epoch >= decay_step:
                cur_decay = cur_decay * opt_cfg.LR_DECAY
        return max(cur_decay, opt_cfg.LR_CLIP / opt_cfg.LR)

    if opt_cfg.get('SCHEDULER', None) == 'cosine':
        scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            optimizer,
            T_0=2 * len(dataloader),
            T_mult=1,
            eta_min=max(1e-2 * opt_cfg.LR, 1e-6),
            last_epoch=-1,
        )
    elif opt_cfg.get('SCHEDULER', None) == 'lambdaLR':
        scheduler = lr_sched.LambdaLR(optimizer, lr_lbmd, last_epoch=last_epoch)
    elif opt_cfg.get('SCHEDULER', None) == 'linearLR':
        total_iters = total_iters_each_epoch * total_epochs
        scheduler = lr_sched.LinearLR(optimizer, start_factor=1.0, end_factor=opt_cfg.LR_CLIP / opt_cfg.LR, total_iters=total_iters, last_epoch=last_epoch)
    else:
        scheduler = None

    return scheduler


def main():
    args, cfg = parse_config()
    if args.launcher == 'none':
        dist_train = False
        total_gpus = 1
        args.without_sync_bn = True
    else:
        if args.local_rank is None:
            args.local_rank = int(os.environ.get('LOCAL_RANK', '0'))
        total_gpus, cfg.LOCAL_RANK = getattr(common_utils, 'init_dist_%s' % args.launcher)(
            args.tcp_port, args.local_rank, backend='nccl'
        )
        dist_train = True

    if args.batch_size is None:
        args.batch_size = cfg.OPTIMIZATION.BATCH_SIZE_PER_GPU
    else:
        assert args.batch_size % total_gpus == 0, 'Batch size should match the number of gpus'
        args.batch_size = args.batch_size // total_gpus

    args.epochs = cfg.OPTIMIZATION.NUM_EPOCHS if args.epochs is None else args.epochs

    if args.fix_random_seed:
        common_utils.set_random_seed(666)

    output_dir = Path("/code/jjiang23/csc587/KimchiVision/output/")
    ckpt_dir = output_dir / 'ckpt'
    output_dir.mkdir(parents=True, exist_ok=True)
    ckpt_dir.mkdir(parents=True, exist_ok=True)

    log_file = output_dir / ('log_train_%s.txt' % datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    logger = common_utils.create_logger(log_file, rank=cfg.LOCAL_RANK)

    # log to file
    logger.info('**********************Start logging**********************')
    gpu_list = os.environ['CUDA_VISIBLE_DEVICES'] if 'CUDA_VISIBLE_DEVICES' in os.environ.keys() else 'ALL'
    logger.info('CUDA_VISIBLE_DEVICES=%s' % gpu_list)

    if dist_train:
        logger.info('total_batch_size: %d' % (total_gpus * args.batch_size))
    for key, val in vars(args).items():
        logger.info('{:16} {}'.format(key, val))
    log_config_to_file(cfg, logger=logger)
    if cfg.LOCAL_RANK == 0:
        os.system('cp %s %s' % (args.cfg_file, output_dir))
    tb_log = SummaryWriter(log_dir=str(output_dir / 'tensorboard')) if cfg.LOCAL_RANK == 0 else None

    train_set, train_loader, train_sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG,
        batch_size=args.batch_size,
        dist=dist_train, workers=args.workers,
        logger=logger,
        training=True,
        merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
        total_epochs=args.epochs,
        add_worker_init_fn=args.add_worker_init_fn,
    )

    model = TrajectoryLSTM()
    if not args.without_sync_bn:
        model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)
    model.cuda()

    optimizer = build_optimizer(model, cfg.OPTIMIZATION)

    # load checkpoint if it is possible
    start_epoch = it = 0
    last_epoch = -1

    if args.pretrained_model is not None:
        model.load_params_from_file(filename=args.pretrained_model, to_cpu=dist_train, logger=logger)

    if args.ckpt is not None:
        it, start_epoch = model.load_params_with_optimizer(args.ckpt, to_cpu=dist_train, optimizer=optimizer,
                                                           logger=logger)
        last_epoch = start_epoch + 1
    else:
        ckpt_list = glob.glob(str(ckpt_dir / '*.pth'))
        if len(ckpt_list) > 0:
            ckpt_list.sort(key=os.path.getmtime)
            while len(ckpt_list) > 0:
                basename = os.path.basename(ckpt_list[-1])
                if basename == 'best_model.pth':
                    ckpt_list = ckpt_list[:-1]
                    continue

                try:
                    it, start_epoch = model.load_params_with_optimizer(
                        ckpt_list[-1], to_cpu=dist_train, optimizer=optimizer, logger=logger
                    )
                    last_epoch = start_epoch + 1
                    break
                except:
                    ckpt_list = ckpt_list[:-1]

    scheduler = build_scheduler(
        optimizer, train_loader, cfg.OPTIMIZATION, total_epochs=args.epochs,
        total_iters_each_epoch=len(train_loader), last_epoch=last_epoch
    )

    model.train()  # before wrap to DistributedDataParallel to support to fix some parameters

    if dist_train:
        model = nn.parallel.DistributedDataParallel(model, device_ids=[cfg.LOCAL_RANK % torch.cuda.device_count()], find_unused_parameters=True)
    logger.info(model)
    num_total_params = sum([x.numel() for x in model.parameters()])
    logger.info(f'Total number of parameters: {num_total_params}')

    test_set, test_loader, sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG,
        batch_size=args.batch_size,
        dist=dist_train, workers=args.workers, logger=logger, training=False
    )

    eval_output_dir = Path("/code/jjiang23/csc587/KimchiVision/output/eval")
    eval_output_dir.mkdir(parents=True, exist_ok=True)

    # -----------------------start training---------------------------
    logger.info('**********************Start training %s/%s(%s)**********************'
                )
    train_model(
        model,
        optimizer,
        train_loader,
        optim_cfg=cfg.OPTIMIZATION,
        start_epoch=start_epoch,
        total_epochs=args.epochs,
        start_iter=it,
        rank=cfg.LOCAL_RANK,
        ckpt_save_dir=ckpt_dir,
        train_sampler=train_sampler,
        ckpt_save_interval=args.ckpt_save_interval,
        max_ckpt_save_num=args.max_ckpt_save_num,
        merge_all_iters_to_one_epoch=args.merge_all_iters_to_one_epoch,
        tb_log=tb_log,
        scheduler=scheduler,
        logger=logger,
        eval_output_dir=eval_output_dir,
        test_loader=test_loader if not args.not_eval_with_train else None,
        cfg=cfg, dist_train=dist_train, logger_iter_interval=args.logger_iter_interval,
        ckpt_save_time_interval=args.ckpt_save_time_interval
    )

    logger.info('**********************End training %s/%s(%s)**********************\n\n\n'
                )


    logger.info('**********************Start evaluation %s/%s(%s)**********************' 
                )

    eval_output_dir = output_dir / 'eval' / 'eval_with_train'
    eval_output_dir.mkdir(parents=True, exist_ok=True)
    args.start_epoch = max(args.epochs - 0, 0)  # Only evaluate the last 10 epochs
    cfg.DATA_CONFIG.SAMPLE_INTERVAL.val = 1

    test_set, test_loader, sampler = build_dataloader(
        dataset_cfg=cfg.DATA_CONFIG,
        batch_size=args.batch_size,
        dist=dist_train, workers=args.workers, logger=logger, training=False
    )

    from test import repeat_eval_ckpt, eval_single_ckpt
    repeat_eval_ckpt(
        model.module if dist_train else model,
        test_loader, args, eval_output_dir, logger, ckpt_dir,
        dist_test=dist_train
    )

    logger.info('**********************End evaluation %s/%s(%s)**********************' 
                )


if __name__ == '__main__':
    main()



2025-06-09 21:40:22,528   INFO  **********************Start logging**********************
2025-06-09 21:40:22,528   INFO  **********************Start logging**********************
2025-06-09 21:40:22,528   INFO  **********************Start logging**********************
2025-06-09 21:40:22,530   INFO  CUDA_VISIBLE_DEVICES=1
2025-06-09 21:40:22,530   INFO  CUDA_VISIBLE_DEVICES=1
2025-06-09 21:40:22,530   INFO  CUDA_VISIBLE_DEVICES=1
2025-06-09 21:40:22,532   INFO  batch_size       32
2025-06-09 21:40:22,532   INFO  batch_size       32
2025-06-09 21:40:22,532   INFO  batch_size       32
2025-06-09 21:40:22,533   INFO  workers          4
2025-06-09 21:40:22,533   INFO  workers          4
2025-06-09 21:40:22,533   INFO  workers          4
2025-06-09 21:40:22,535   INFO  merge_all_iters_to_one_epoch False
2025-06-09 21:40:22,535   INFO  merge_all_iters_to_one_epoch False
2025-06-09 21:40:22,535   INFO  merge_all_iters_to_one_epoch False
2025-06-09 21:40:22,536   INFO  epochs           1
2025

2025-06-09 21:40:22,561   INFO  max_waiting_mins 0
2025-06-09 21:40:22,562   INFO  start_epoch      0
2025-06-09 21:40:22,562   INFO  start_epoch      0
2025-06-09 21:40:22,562   INFO  start_epoch      0
2025-06-09 21:40:22,564   INFO  save_to_file     False
2025-06-09 21:40:22,564   INFO  save_to_file     False
2025-06-09 21:40:22,564   INFO  save_to_file     False
2025-06-09 21:40:22,565   INFO  not_eval_with_train False
2025-06-09 21:40:22,565   INFO  not_eval_with_train False
2025-06-09 21:40:22,565   INFO  not_eval_with_train False
2025-06-09 21:40:22,567   INFO  logger_iter_interval 50
2025-06-09 21:40:22,567   INFO  logger_iter_interval 50
2025-06-09 21:40:22,567   INFO  logger_iter_interval 50
2025-06-09 21:40:22,568   INFO  ckpt_save_time_interval 300
2025-06-09 21:40:22,568   INFO  ckpt_save_time_interval 300
2025-06-09 21:40:22,568   INFO  ckpt_save_time_interval 300
2025-06-09 21:40:22,569   INFO  pretrained_model None
2025-06-09 21:40:22,569   INFO  pretrained_model None
2

This itteration loss:  2306.97900390625
This itteration loss:  1985.9320068359375
This itteration loss:  2384.45947265625
This itteration loss:  2944.831787109375
This itteration loss:  2657.96533203125
This itteration loss:  1393.2159423828125
This itteration loss:  2073.966064453125
This itteration loss:  2123.84912109375
This itteration loss:  2241.537109375
This itteration loss:  1912.5426025390625
This itteration loss:  1811.248291015625
This itteration loss:  1839.11328125
This itteration loss:  1911.8602294921875
This itteration loss:  2156.53369140625
This itteration loss:  2110.66015625
This itteration loss:  2044.5360107421875
This itteration loss:  1735.2515869140625
This itteration loss:  2149.593994140625
This itteration loss:  1820.7718505859375
This itteration loss:  2211.1591796875
This itteration loss:  1567.8333740234375
This itteration loss:  1929.1478271484375
This itteration loss:  1721.2154541015625
This itteration loss:  2517.103515625
This itteration loss:  2717

2025-06-09 21:44:22,196   INFO  epoch: 0/1, acc_iter=50, cur_iter=49/7607, batch_size=32, iter_cost=4.37s, time_cost(epoch): 03:38/9:09:55, time_cost(all): 03:39/9:09:55, loss=896.815, lr=0.0001
2025-06-09 21:44:22,196   INFO  epoch: 0/1, acc_iter=50, cur_iter=49/7607, batch_size=32, iter_cost=4.37s, time_cost(epoch): 03:38/9:09:55, time_cost(all): 03:39/9:09:55, loss=896.815, lr=0.0001
2025-06-09 21:44:22,196   INFO  epoch: 0/1, acc_iter=50, cur_iter=49/7607, batch_size=32, iter_cost=4.37s, time_cost(epoch): 03:38/9:09:55, time_cost(all): 03:39/9:09:55, loss=896.815, lr=0.0001


This itteration loss:  896.8146362304688
This itteration loss:  1075.9674072265625
This itteration loss:  883.4511108398438
This itteration loss:  822.7139282226562
This itteration loss:  806.2197265625
This itteration loss:  981.6978759765625
This itteration loss:  804.0809936523438
This itteration loss:  671.5686645507812
This itteration loss:  855.9968872070312
This itteration loss:  746.6047973632812
This itteration loss:  698.5769653320312
This itteration loss:  1059.921875
This itteration loss:  746.9793701171875
This itteration loss:  663.0576171875
This itteration loss:  924.6609497070312
This itteration loss:  660.3490600585938
This itteration loss:  385.2032165527344
This itteration loss:  634.4189453125
This itteration loss:  646.2720947265625
This itteration loss:  523.1420288085938
This itteration loss:  589.9116821289062
This itteration loss:  728.7267456054688
This itteration loss:  608.7325439453125
This itteration loss:  632.9767456054688


...

This itteration loss:  187.37979125976562
This itteration loss:  279.7096252441406
This itteration loss:  176.489990234375
This itteration loss:  208.8330841064453
This itteration loss:  174.7793731689453
This itteration loss:  164.35145568847656
This itteration loss:  181.9660186767578
This itteration loss:  312.4028625488281
This itteration loss:  225.43661499023438
This itteration loss:  164.8878936767578
This itteration loss:  219.1406707763672
This itteration loss:  204.4886932373047
This itteration loss:  202.0963592529297
This itteration loss:  198.9890594482422
This itteration loss:  164.1947784423828
This itteration loss:  188.30795288085938
This itteration loss:  178.42501831054688
This itteration loss:  205.00376892089844
This itteration loss:  204.26731872558594
This itteration loss:  173.61143493652344
This itteration loss:  156.95753479003906
This itteration loss:  184.86581420898438
This itteration loss:  214.72303771972656
This itteration loss:  187.2571258544922
This i

2025-06-09 21:55:46,474   INFO  Save latest model to /code/jjiang23/csc587/KimchiVision/output/ckpt/latest_model
2025-06-09 21:55:46,474   INFO  Save latest model to /code/jjiang23/csc587/KimchiVision/output/ckpt/latest_model
2025-06-09 21:55:46,474   INFO  Save latest model to /code/jjiang23/csc587/KimchiVision/output/ckpt/latest_model


This itteration loss:  174.5293731689453
This itteration loss:  182.82687377929688
This itteration loss:  251.51942443847656
This itteration loss:  175.22618103027344
This itteration loss:  216.09912109375


epochs:   0%|          | 0/1 [15:29<?, ?it/s]


KeyboardInterrupt: 